# Overview

This notebook was created to enable me to try out my tensorflow skills and to apply my understanding of Neural Networks and CNN on a problem which I could work on myself without having to use a pretrained model.

# Setup

In [ ]:
IMG_SIZE = [28,28,1]
NUMBER_OF_CLASSES = 10

In [ ]:
train_df = pd.read_csv('/data/train.csv')
test_df = pd.read_csv('/data/test.csv')

# Functions and Parameters

In [ ]:
def unflatten_image(df_row):
    if len(df_row) == 785:
        flat_array = df_row.iloc[1:].values / 255
    else:
        flat_array = df_row.values / 255
    reshapped_array = flat_array.reshape(28,28)
    return reshapped_array

def display_number(reshapped_array, label=None):
    if label:
        plt.title(f"Label: {label}")
    plt.axis('off')
    plt.imshow(reshapped_array, cmap='gray')
    
# def unflatten_and_display(df_row):
#     reshapped_array = unflatten_image(df_row)
#     display_number(reshapped_array, df_row['label'])

In [ ]:
def read_in_data_and_convert_to_format(df):
    X_df = df
    if 'label' in df.columns:
        X_df = df.iloc[:, 1:]
    unflattened_all_images_df = X_df.apply(unflatten_image, axis=1)
    unflattened_all_images_array = unflattened_all_images_df.to_numpy()
    X = np.stack(unflattened_all_images_array, axis=0)
    return X

In [ ]:
def output_submission(predictions):
    submission_df = pd.DataFrame(columns=['ImageId', 'Label'])
    submission_df['Label'] = predictions
    submission_df['ImageId'] = submission_df.index + 1
    submission_df.to_csv('/kaggle/working/submission.csv', index=False)
    return submission_df

## Read in Data

In [ ]:
X_train = read_in_data_and_convert_to_format(train_df)
y_train = train_df['label']

In [ ]:
X_test = read_in_data_and_convert_to_format(test_df)

# EDA

In [ ]:
display_number(X_train[9], y_train[9])

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
BATCH_SIZE

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(IMG_SIZE)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPool2D(2))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPool2D(2))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPool2D(2))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(2,2), activation='relu'))
model.add(MaxPool2D())
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(Dense(NUMBER_OF_CLASSES, activation='linear'))

In [ ]:
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer=Adam(learning_rate=0.01), metrics=['sparse_categorical_accuracy'])

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=100)

In [ ]:
model.save("model_file.keras")

In [ ]:
history.history

In [ ]:
output = model.predict(X_test)

In [ ]:
predicted_classes = np.argmax(output, axis=1)
predicted_classes

In [ ]:
submission_df = output_submission(predicted_classes)
submission_df

In [ ]:
model.summary()